In [5]:
import pandas as pd
import requests

# 1. 동탄역 정차 노선 목록
> 교통카드 빅데이터 시스템

In [8]:
df_east = pd.read_csv('./GTX_동탄역_data/교통카드 빅데이터/동탄역(동측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')
df_west = pd.read_csv('./GTX_동탄역_data/교통카드 빅데이터/동탄역(서측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')

In [66]:
df_east

,정류장명,정류장번호,노선,기종점,일자,이용자유형,04(승차),04(하차),05(승차),05(하차),...,23(하차),00(승차),00(하차),01(승차),01(하차),02(승차),02(하차),03(승차),03(하차),Unnamed: 54
0,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,일반인,3,0,7,0,...,7,0,15,0,0,0,0,0,0,NaN
1,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,어린이,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,청소년,0,0,0,0,...,1,0,1,0,0,0,0,0,0,NaN
3,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,경로,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,장애인,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,동탄역(동측),55399,H9,방교동일반산업단지 - 방교동일반산업단지,2024-05,장애인,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
198,동탄역(동측),55399,H9,방교동일반산업단지 - 방교동일반산업단지,2024-05,국가유공자,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
199,동탄역(동측),55399,화성똑버스01,동탄역(동측) - 동탄역(서측),2024-05,일반인,0,0,52,5,...,116,0,0,0,0,0,0,0,0,NaN
200,동탄역(동측),55399,화성똑버스01,동탄역(동측) - 동탄역(서측),2024-05,어린이,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [12]:
#동탄역(동측+서측)에 정차하는 노선 목록 확인
east = df_east['노선'].unique().tolist()
west = df_west['노선'].unique().tolist()
bus_lane = (east+west)

In [14]:
len(bus_lane)

62

In [13]:
#노선수
bus_lane

['1000',
 '17',
 '17-1',
 '19-3',
 '19-5(A)',
 '19-5(B)',
 '200',
 '203',
 '205',
 '7-1A',
 '701A',
 '709',
 '73',
 '81',
 '8202',
 '9',
 '90',
 '94',
 'G6010',
 'H1',
 'H100',
 'H101',
 'H102',
 'H11',
 'H12',
 'H15',
 'H17',
 'H18',
 'H2',
 'H20',
 'H24',
 'H4',
 'H6(A)',
 'H65',
 'H67',
 'H9',
 '화성똑버스01',
 '19-3',
 '200',
 '203',
 '205',
 '39',
 '45',
 '7-1A',
 '701A',
 '709-1',
 '9',
 '90',
 'G6010',
 'H1',
 'H100',
 'H101',
 'H102',
 'H11',
 'H12',
 'H17',
 'H18',
 'H2',
 'H24',
 'H4',
 'H6(B)',
 'H65']

# 2. 세대 현황
> 행정안전부

In [34]:
toll = pd.read_excel('C:/Users/OneYoung/Documents/GitHub/GTX-A-Bus-lane-optimization/GTX_동탄역_data/행정안전부/행안부_202405_202405_주민등록인구및세대현황_월간.xlsx')
# 특정 행을 컬럼으로 설정하기
toll.columns = toll.iloc[1]

# 불필요한 행 제거 (첫 두 행 제거)
toll = toll[2:]

# 인덱스 리셋 (기존 인덱스는 드롭)
toll = toll.reset_index(drop=True)

toll.head(4)

c:\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1,행정기관코드,행정기관,총 거주자수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,4159000000,경기도 경기도 화성시,"949,720","407,372",2.33,"492,894","456,826",1.08
1,4159025300,경기도 화성시 봉담읍,"98,700","41,334",2.39,"49,906","48,794",1.02
2,4159025600,경기도 화성시 우정읍,"17,275","9,307",1.86,"9,508","7,767",1.22
3,4159025900,경기도 화성시 향남읍,"85,845","40,355",2.13,"46,143","39,702",1.16


In [35]:
toll['행정기관'].unique()

array(['경기도 경기도 화성시 ', '경기도 화성시 봉담읍', '경기도 화성시 우정읍', '경기도 화성시 향남읍',
       '경기도 화성시 남양읍', '경기도 화성시 매송면', '경기도 화성시 비봉면', '경기도 화성시 마도면',
       '경기도 화성시 송산면', '경기도 화성시 서신면', '경기도 화성시 팔탄면', '경기도 화성시 장안면',
       '경기도 화성시 양감면', '경기도 화성시 정남면', '경기도 화성시 새솔동', '경기도 화성시 진안동',
       '경기도 화성시 병점1동', '경기도 화성시 병점2동', '경기도 화성시 반월동', '경기도 화성시 기배동',
       '경기도 화성시 화산동', '경기도 화성시 동탄1동', '경기도 화성시 동탄2동', '경기도 화성시 동탄3동',
       '경기도 화성시 동탄4동', '경기도 화성시 동탄5동', '경기도 화성시 동탄6동', '경기도 화성시 동탄7동',
       '경기도 화성시 동탄8동', '경기도 화성시 동탄9동'], dtype=object)

# 3. 버스 정류장 이용량 지표
> 교통카드 빅데이터 시스템

In [19]:
df = pd.read_excel('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/발생량_도착량_이용량 지표(목적통행량)_20240617.xlsx')
df

,시간대:,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,합계,Unnamed: 7,04,Unnamed: 9,...,23,Unnamed: 47,00,Unnamed: 49,01,Unnamed: 51,02,Unnamed: 53,03,Unnamed: 55
0,시도코드,시도,시군구,읍면동,이용자유형,월,발생량,도착량,발생량,도착량,...,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량
1,합계,0,0,0,0,0,2057675,2034813,891,65,...,22646,69613,7616,33096,3470,6588,1838,1933,160,375
2,41,경기도,화성시,금곡동,경로,202405,353,410,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,국가유공자,202405,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,어린이,202405,10,10,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0,0,0,0,국가유공자,202405,104,91,0,0,...,0,4,1,3,0,0,0,0,0,0
82,0,0,0,0,어린이,202405,3513,4129,0,0,...,0,16,0,2,0,0,0,0,0,0
83,0,0,0,0,일반인,202405,136193,114799,74,10,...,1489,5006,585,2564,352,387,163,71,12,13
84,0,0,0,0,장애인,202405,676,645,0,0,...,2,10,5,7,1,0,4,1,0,0


# 4. 정차 노선수
>교통카드 빅데이터 시스템

In [21]:
df = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/노선·정류장 지표(정류장별 정차 노선수)_20240617.csv', encoding = 'euc-kr')
df

,시도,시군구,읍면동,정류장번호,정류장명,노선수,정차 노선 명
0,경기도,화성시,오산동,55399,동탄역(동측),36,"1000, 17, 17-1, 19-3, 19-5(A), 19-5(B), 200, 2..."
1,경기도,화성시,오산동,55398,동탄역(서측),26,"19-3, 200, 203, 205, 39, 45, 7-1A, 701A, 709-1..."


# 5. 버스 정류장 주소 데이터
> 공공데이터 포털

In [24]:
df = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/공공데이터포털/(수정)경기도 화성시 버스 정류장 주소.csv')
df

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,도로명 주소
0,GGB233000459,석포3리,37.120617,126.830883,2023-10-16,36886.0,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 115-1
1,GGB233000460,석포2리,37.129267,126.837217,2023-10-16,37438.0,31240,경기도 화성시,경기,경기 화성시 팔탄면 월문리 248
2,GGB233000461,석포4리,37.130050,126.839583,2023-10-16,36885.0,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 426-2
3,GGB233000462,화당리,37.122017,126.864833,2023-10-16,36884.0,31240,경기도 화성시,경기,경기 화성시 팔탄면 월문리 64-2
4,GGB233000463,월문초등학교,37.117900,126.871517,2023-10-16,37437.0,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 415-3
...,...,...,...,...,...,...,...,...,...,...
3021,GGB277103853,송산마도IC(경유),37.209900,126.759633,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 송산면 삼존리 803-1
3022,GGB277103854,송산마도IC(경유),37.209383,126.759967,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 송산면 삼존리 806
3023,GGB277103887,조암IC(경유),37.118633,126.818267,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 향남읍 행정리 472-1
3024,GGB277103888,조암IC(경유),37.118417,126.818867,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 우정읍 주곡리 837-39


# 6. 버스 정류장 배차 간격 데이터
> 공공데이터 포털

In [61]:
df = pd.read_excel('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/공공데이터포털/공공데이터포털_경기도 화성시_마을버스노선현황_20230905.xlsx')
df

,운수사,노선번호,기점,종점,출퇴근배차,평일배차,주말배차,주중상행첫차,주중상행막차,주말상행첫차,주말상행막차,주중하행첫차,주중하행막차,주말하행첫차,주말하행막차
0,금오운수,17,병점역사거리,동탄역(동측),15,15~20,25~35,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
1,금오운수,17-1,병점역후문,동탄역,40,40~60,150~160,06:00:00,22:30:00,06:00:00,22:30:00,08:20:00,17:10:00,-,-
2,금오운수,27,병점역사거리,한림대성심병원,8,8~12,15~20,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
3,금오운수,27-2,병점역후문,나루고교,97,97~117,-,07:43:00,16:11:00,-,-,08:33:00,17:09:00,-,-
4,다원교통주식회사,22,수현초.수현중,어천역,15,15~20,15~20,05:40:00,20:50:00,05:40:00,20:50:00,05:06:00,21:15:00,06:05:00,21:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,화성도시공사,H50-7,화성의과학대학교,사강복지회관,90,90~110,90~110,06:00:00,22:00:00,06:40:00,22:40:00,06:40:00,22:40:00,06:40:00,22:40:00
168,화성도시공사,H50-9,화성의과학대학교,화성시청서희스타힐스1단지,90,90~110,90~110,05:30:00,21:20:00,05:30:00,21:20:00,06:00:00,21:50:00,06:00:00,21:50:00
169,화성도시공사,H51,전곡항,백미리사랑방,100,100~120,100~120,06:00:00,22:00:00,06:00:00,22:00:00,06:40:00,22:40:00,06:40:00,22:40:00
170,화성도시공사,H52,전곡항,궁평유원지,110,110~140,110~140,06:00:00,21:50:00,06:00:00,21:50:00,06:45:00,22:35:00,06:45:00,22:35:00


# 배차 간격의 평균 값 계산
> 마을버스노선현황의 '출퇴근배차간격'과 동탄역을 정차하는 노선 62개에 대해서 매핑

In [62]:
# '노선번호' 컬럼을 string으로 변환
df['노선번호'] = df['노선번호'].astype(str)

# 제외할 한글 포함 데이터 제거
df = df[~df['노선번호'].str.contains(r'[ㄱ-힣]+')]

df['노선번호'].unique()

array(['17', '17-1', '27', '27-2', '22', '22-2', '50-1', '50-5A', '50-5B',
       '6-3', '6-4', '50-2', '50-2(A)', '22-4(A)', '22-4(B)', '22-5',
       '22-6', '50-8', '22-1', '20-5', '11', '11-2', '12', '13', 'H13(A)',
       'H13(B)', 'H25', '100', '100(A)', '35-1(A)', '35-1(B)', '35-2',
       '38', '38-1', '19', '19-1', '19-2', '19-3', '19-5(A)', '19-5(B)',
       '39', 'H11', 'H12', 'H16', 'H17', 'H9', '6-1', '6-2', '31', '31-1',
       '31-2', 'E1', '20-4(A)', '3-2', '333', '50', '50-3', '7-1', '709',
       '709-1', '76', '77', '78', '79', '80', '81', '9', '9-1', 'H4',
       'H5', 'H6(A)', 'H6(B)', '5', '6', 'H14(A)', 'H14(B)', 'H15', 'H19',
       'H20', '10-2', '10-3', '10-3A', '10', '10-1', '10-5', '11-1',
       '12-1', '13-1', '13-1A', '13-2', '13-3', '13-3(A)', '13-4',
       '13-4(A)', '36', '36A', '36-1', '36-2', '36-2A', '36-3', '37',
       '39A', '39-1', '39-2', 'H7', 'H7-1', 'H1', 'H10-4', 'H10-6',
       'H10-7', 'H10-8', 'H13-5', 'H13-7', 'H2', 'H50', 'H50-3', 'H5

In [63]:
#동탄역 정차 노선 번호 리스트
print(bus_lane)

['1000', '17', '17-1', '19-3', '19-5(A)', '19-5(B)', '200', '203', '205', '7-1A', '701A', '709', '73', '81', '8202', '9', '90', '94', 'G6010', 'H1', 'H100', 'H101', 'H102', 'H11', 'H12', 'H15', 'H17', 'H18', 'H2', 'H20', 'H24', 'H4', 'H6(A)', 'H65', 'H67', 'H9', '화성똑버스01', '19-3', '200', '203', '205', '39', '45', '7-1A', '701A', '709-1', '9', '90', 'G6010', 'H1', 'H100', 'H101', 'H102', 'H11', 'H12', 'H17', 'H18', 'H2', 'H24', 'H4', 'H6(B)', 'H65']


In [ ]:
# 리스트와 일치하는 노선번호만 필터링
filtered_routes = df[df['노선번호'].isin(bus_lane)]
filtered_routes = filtered_routes.reset_index(drop=True)

# 115index의 값(30~30)을 30으로 바꿔줌
filtered_routes['출퇴근배차'][16] = "30"

# Converting '출퇴근배차' to numeric
filtered_routes["출퇴근배차"] = pd.to_numeric(filtered_routes["출퇴근배차"])

# Calculating the mean
mean_value = filtered_routes["출퇴근배차"].mean()

In [67]:
filtered_routes

,운수사,노선번호,기점,종점,출퇴근배차,평일배차,주말배차,주중상행첫차,주중상행막차,주말상행첫차,주말상행막차,주중하행첫차,주중하행막차,주말하행첫차,주말하행막차
0,금오운수,17,병점역사거리,동탄역(동측),15,15~20,25~35,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
1,금오운수,17-1,병점역후문,동탄역,40,40~60,150~160,06:00:00,22:30:00,06:00:00,22:30:00,08:20:00,17:10:00,-,-
2,산척여객,19-3,LH1단지.한양수자인,병점역사거리,40,40~50,60~70,05:15:00,21:20:00,05:15:00,21:20:00,06:00:00,22:15:00,06:00:00,22:15:00
3,산척여객,19-5(A),장지동농협,동탄역(동측),50,50~60,50~60,05:30:00,20:48:00,05:30:00,20:48:00,06:10:00,21:28:00,06:10:00,21:28:00
4,산척여객,19-5(B),장지동농협,병점역사거리,65,65~115,65~115,05:30:00,19:10:00,05:30:00,19:10:00,07:12:00,20:52:00,07:12:00,20:52:00
5,산척여객,39,동탄 물류단지,동탄역(서측),80,80~100,80~100,05:50:00,19:35:00,05:50:00,19:35:00,06:30:00,20:15:00,06:30:00,20:15:00
6,산척여객,H11,방아다리마을,현대트랜시스,20,20~30,20~30,05:30:00,21:00:00,05:30:00,21:00:00,06:35:00,22:00:00,06:35:00,22:00:00
7,산척여객,H12,장지동농협,신영통현대타운.두산위브,20,20~50,20~50,05:15:00,20:40:00,05:15:00,20:40:00,06:40:00,22:05:00,06:40:00,22:05:00
8,산척여객,H17,장지동농협,오산역,30,30~34,30~34,05:20:00,20:15:00,05:20:00,20:15:00,05:20:00,21:50:00,05:20:00,21:50:00
9,산척여객,H9,방교동일반산업단지,동탄역(동측),15,15~20,15~20,05:30:00,21:35:00,05:30:00,21:35:00,06:05:00,22:10:00,06:05:00,22:10:00


In [65]:
#출퇴근배차간격 평균 시간(분)
mean_value

np.float64(32.27272727272727)

# 7. 교통량

In [69]:
df = pd.read_excel('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/발생량_도착량_이용량 지표(목적통행량)_20240617.xlsx')
df

,시간대:,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,합계,Unnamed: 7,04,Unnamed: 9,...,23,Unnamed: 47,00,Unnamed: 49,01,Unnamed: 51,02,Unnamed: 53,03,Unnamed: 55
0,시도코드,시도,시군구,읍면동,이용자유형,월,발생량,도착량,발생량,도착량,...,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량
1,합계,0,0,0,0,0,2057675,2034813,891,65,...,22646,69613,7616,33096,3470,6588,1838,1933,160,375
2,41,경기도,화성시,금곡동,경로,202405,353,410,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,국가유공자,202405,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,어린이,202405,10,10,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0,0,0,0,국가유공자,202405,104,91,0,0,...,0,4,1,3,0,0,0,0,0,0
82,0,0,0,0,어린이,202405,3513,4129,0,0,...,0,16,0,2,0,0,0,0,0,0
83,0,0,0,0,일반인,202405,136193,114799,74,10,...,1489,5006,585,2564,352,387,163,71,12,13
84,0,0,0,0,장애인,202405,676,645,0,0,...,2,10,5,7,1,0,4,1,0,0
